In [22]:
# Load the TensorBoard notebook extension.

%reload_ext tensorboard
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras import initializers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from tensorflow.keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pathlib
import pickle
import pathlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from datetime import datetime
import seaborn as sns
import tensorboard
tensorboard.__version__

'2.4.0'

In [3]:
def huber_loss(y_true, y_pred, clip_delta=1.0):
        error = y_true - y_pred
        cond  = tf.keras.backend.abs(error) < clip_delta
        squared_loss = 0.5 * tf.keras.backend.square(error)
        linear_loss  = clip_delta * (tf.keras.backend.abs(error) - 0.5 * clip_delta)
        return tf.where(cond, squared_loss, linear_loss)
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

### 1) Read dataset original

In [45]:
BaseData = pd.read_csv('./Data/summer_limited.csv')
BaseData = BaseData.drop(columns = ['Unnamed: 0'])
# Copy Data for AirTemp & AbsHum scenarios before performing operation on AirTemp & AbsHum for BaseData
AirTempMinus1 = BaseData.copy()
AbsHumMinus1 = BaseData.copy()
AllScen = BaseData.copy()
# Perform operation on AirTemp & AbsHum then Copy Data to Shade Scenarios
BaseData['Air Temp Squared'] = BaseData['Air Temp']**2
BaseData['Abs Hum Squared'] = BaseData['Abs Hum']**2
BaseData['Air Temp x Abs Hum'] = BaseData['Air Temp']*BaseData['Abs Hum']
WestShade25 = BaseData.copy()
EastShade25 = BaseData.copy()
NorthShade25 = BaseData.copy()
SouthShade25 = BaseData.copy()
AllShade25 = BaseData.copy()
# Perform operation for shade scenarios 
WestShade25.loc[WestShade25['Shade West'] < 1, 'Shade West'] = WestShade25['Shade West'] + WestShade25['Shade West']*0.25
EastShade25.loc[EastShade25['Shade East'] < 1, 'Shade East'] = EastShade25['Shade East'] + EastShade25['Shade East']*0.25
NorthShade25.loc[NorthShade25['Shade North'] < 1, 'Shade North'] = NorthShade25['Shade North'] + NorthShade25['Shade North']*0.25
SouthShade25.loc[SouthShade25['Shade South'] < 1, 'Shade South'] = SouthShade25['Shade South'] + SouthShade25['Shade South']*0.25

AllShade25.loc[AllShade25['Shade West'] < 1, 'Shade West'] = AllShade25['Shade West'] + AllShade25['Shade West']*0.25
AllShade25.loc[AllShade25['Shade East'] < 1, 'Shade East'] = AllShade25['Shade East'] + AllShade25['Shade East']*0.25
AllShade25.loc[AllShade25['Shade North'] < 1, 'Shade North'] = AllShade25['Shade North'] + AllShade25['Shade North']*0.25
AllShade25.loc[AllShade25['Shade South'] < 1, 'Shade South'] = AllShade25['Shade South'] + AllShade25['Shade South']*0.25
# Perform operation for AirTempMinus1, AbsHumMinus1, and AllScenarios
AirTempMinus1['Air Temp'] = AirTempMinus1['Air Temp'] - 1
AirTempMinus1['Air Temp Squared'] = AirTempMinus1['Air Temp']**2
AirTempMinus1['Abs Hum Squared'] = AirTempMinus1['Abs Hum']**2
AirTempMinus1['Air Temp x Abs Hum'] = AirTempMinus1['Air Temp']*AirTempMinus1['Abs Hum']

AbsHumMinus1['Abs Hum'] = AbsHumMinus1['Abs Hum'] - 1
AbsHumMinus1['Air Temp Squared'] = AbsHumMinus1['Air Temp']**2
AbsHumMinus1['Abs Hum Squared'] = AbsHumMinus1['Abs Hum']**2
AbsHumMinus1['Air Temp x Abs Hum'] = AbsHumMinus1['Air Temp']*AbsHumMinus1['Abs Hum']

AllScen['Air Temp'] = AllScen['Air Temp'] - 1
AllScen['Abs Hum'] = AllScen['Abs Hum'] - 1
AllScen['Air Temp Squared'] = AllScen['Air Temp']**2
AllScen['Abs Hum Squared'] = AllScen['Abs Hum']**2
AllScen['Air Temp x Abs Hum'] = AllScen['Air Temp']*AllScen['Abs Hum']
AllScen.loc[AllScen['Shade West'] < 1, 'Shade West'] = AllScen['Shade West'] + AllScen['Shade West']*0.25
AllScen.loc[AllScen['Shade East'] < 1, 'Shade East'] = AllScen['Shade East'] + AllScen['Shade East']*0.25
AllScen.loc[AllScen['Shade North'] < 1, 'Shade North'] = AllScen['Shade North'] + AllScen['Shade North']*0.25
AllScen.loc[AllScen['Shade South'] < 1, 'Shade South'] = AllScen['Shade South'] + AllScen['Shade South']*0.25


In [53]:

def prepare_data(setData):
    #add extra features
    X = setData[['bldgname',
                  'Air Temp', 
                  'Abs Hum',
                  'Air Temp Squared',
                  'Abs Hum Squared',
                  'Air Temp x Abs Hum',
                  'DSW Top', 
                  'DSW North', 
                  'DSW South', 
                  'DSW East', 
                  'DSW West', 
                  'Shade North', 
                  'Shade East', 
                  'Shade West',
                  'Shade South']]
    columns = ['Air Temp', 'Abs Hum', 'Air Temp Squared',
                'Abs Hum Squared', 'Air Temp x Abs Hum', 'DSW Top',
               'DSW North', 'DSW South', 'DSW East','DSW West',
               'Shade North', 'Shade East', 'Shade West', 'Shade South',]
    Y = setData['CHWTON/SQFT']
    Y = Y.values.reshape(-1, 1)    
    Y = yscaler .fit_transform(Y)
    X = pd.get_dummies(X)
    X[columns] = xscaler.fit_transform(X[columns])
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)
    return X_train, X_test, y_train, y_test

In [47]:
X_train, X_test, y_train, y_test = prepare_data(BaseData)

### 2) Train Model

In [48]:
model = Sequential()
model.add(Dense(30, input_dim=25, activation= 'relu'))
model.add(Dense(40, activation = 'relu'))
model.add(Dense(40, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dropout(0.05))
model.add(Dense(1, activation = 'linear'))
model.compile( loss = 'mse', optimizer = 'adam')
model.summary()
history = model.fit(X_train,y_train, 
                            batch_size=128, 
                            epochs=200, 
                            verbose=0, 
                            validation_split=0.2)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 30)                780       
_________________________________________________________________
dense_16 (Dense)             (None, 40)                1240      
_________________________________________________________________
dense_17 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_18 (Dense)             (None, 50)                2050      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 51        
Total params: 5,761
Trainable params: 5,761
Non-trainable params: 0
____________________________________________________

In [42]:
y_preds = model.predict(X_test)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_preds))
R2_score = metrics.r2_score(y_test, y_preds)
print("The RMSE score for test dataset of ", RMSE)
print("The R2 score for test dataset of " ,R2_score)

The RMSE score for test dataset of  0.11950383593962852
The R2 score for test dataset of  0.986149961710786


## Run below script only once after training. Before running script below, run scripts: 1, 2 and 3(whichever required) , and then run whole cycle again if required.

In [60]:
for bldg in range(len(Buildings)):
    # plot predictions for baseline data data
    date_time = MicroclimateData[['Date_Time',
                            'Month',
                            'Day',
                            'Hour',
                            'Minute']]
    #for baseline results
    yscaler = StandardScaler()
    xscaler = StandardScaler()
    X_train, X_test, y_train, y_test = prepare_data(BaseData)

    comb1 = [X_train, X_test]
    X_train= pd.concat(comb1)
    #     print("old X",X_train)
    Y_preds = model.predict(X_train)
    y_train = np.concatenate((y_train, y_test))
    results_NN = undummify(X_train).join(date_time)
    results_NN['Baseline'] = yscaler.inverse_transform(Y_preds)


    #for new data results
    X_train, X_test, y_train, y_test = prepare_data(EastShade25)
    comb2 = [X_train, X_test]
    X_train = pd.concat(comb2)
    # print("new X",X_train_new)
    Y_preds = model.predict(X_train)
    # print("new result", Y_preds_new)
    results_NN['Predicted with change'] = yscaler.inverse_transform(Y_preds)
#     print(Y_preds_new)
    
    
    # find out for particular day
    month_NN = results_NN[results_NN['Month'] == 6]
    day_NN = month_NN[month_NN['Day'] == 7]
    bldg_NN = day_NN.loc[day_NN['bldgname'] == Buildings[bldg]]
    bldg_NN['Time'] = bldg_NN['Date_Time'].str[11:16]  
    # print(bldg_NN)
    values_NN = bldg_NN.sort_values(by=['Time'])
    values_NN['Model Predictions on Baseline Data']= values_NN['Baseline'] 
#     values_NN['Model predictions after reducing temperature']= values_NN['Predicted with change'] 
    # #plot values_NN
    ax = values_NN.plot(x='Time',y='Model Predictions on Baseline Data',grid = False, figsize = (8,8))
#     values_NN.plot(x='Time',y='Baseline',grid = False, figsize = (8,8),ax=ax)
    values_NN.plot(x='Time',y='Predicted with change',grid = False, figsize = (8,8),ax=ax)
    string = './Data/Plots/sensitivity analysis/7-6/WithShadeMod/East/'+ Buildings[bldg]
    
    plt.savefig(string, bbox_inches='tight')  


IndentationError: unexpected indent (2240484206.py, line 2)